In [1]:
import numpy as np
import cv2
import json

average_frame.png의 카메라 위치와 hs_를 찍은 카메라 위치는 동일. 

이미지가 뒤집혀져있어서 ArUCo의 tvec, rvec을 뒤집어줘야함.  
Camera matrix는 방향과 상관 없어서 뒤집지 않아도 됨. 


In [562]:
def rotation_matrix(alpha, beta, gamma):
    """
    Compute the general 3D rotation matrix for rotations around the X, Y, and Z axes.

    :param alpha: Rotation around the X-axis in radians
    :param beta: Rotation around the Y-axis in radians
    :param gamma: Rotation around the Z-axis in radians
    :return: The combined 3D rotation matrix
    """
    alpha = np.deg2rad(alpha)
    beta = np.deg2rad(beta)
    gamma = np.deg2rad(gamma)
    # Rotation matrix for rotation around the X-axis
    R_x = np.array([[1, 0, 0],
                    [0, np.cos(alpha), -np.sin(alpha)],
                    [0, np.sin(alpha), np.cos(alpha)]])

    # Rotation matrix for rotation around the Y-axis
    R_y = np.array([[np.cos(beta), 0, np.sin(beta)],
                    [0, 1, 0],
                    [-np.sin(beta), 0, np.cos(beta)]])

    # Rotation matrix for rotation around the Z-axis
    R_z = np.array([[np.cos(gamma), -np.sin(gamma), 0],
                    [np.sin(gamma), np.cos(gamma), 0],
                    [0, 0, 1]])

    # Combine the rotations (Z * Y * X)
    R = np.dot(R_z, np.dot(R_y, R_x))
    return R

In [390]:
connections = [(0,1),(1,2),(2,3), (3,4),(4,5),(5,6),
               (4,7),(7,8),(8,9),(9,10),
               (4,11), (11,12), (12,13), (13,14),
               (0, 15), (15,16), (16,17), (17,18),
               (0,19), (19,20), (20,21), (21,22)]

connections_coco3d =[(0,1), (1,2), (0,3), (3,4),
                    (1,9), (9,10), (10,11), 
                    (3,6), (6,7), (7,8),
                    (9,5), (5,6)]

image_width = 1920
image_height = 1080
# Camera intrinsic parameters
# 카메라 매트릭스와 왜곡 계수 (미리 구해둬야 함)
CAMERA_MATRIX_ORG = np.array([[781.42789058, 0., 994.45013811],
                          [0., 778.74286573, 602.69690385],
                          [0., 0., 1.]])
CAMERA_MATRIX = CAMERA_MATRIX_ORG.copy()
CAMERA_MATRIX[0, 2] = image_width - CAMERA_MATRIX_ORG[0,2] # flip x
CAMERA_MATRIX[1, 2] = image_height - CAMERA_MATRIX_ORG[1,2] # flip y
DIST_COEFFS = np.array([[1.87498264e+01, 6.82542530e+00, -1.08541825e-04,
                         -1.58742050e-05, 1.49993131e-01, 1.91587107e+01,
                         1.32224486e+01, 1.15118961e+00, 0., 0., 0., 0.,
                         0., 0.]])
MARKER_LENGTH = 185

In [391]:
pos = json.load(open('./calibration/pose_data.json', "r"))
rvec = np.array(pos['rvecs'])
tvec = np.array(pos['tvecs'])

In [405]:
def vertcal_flip_rvec_tvec(rvec, tvec):
    R_marker_to_camera, _ = cv2.Rodrigues(rvec)
    # Flip matrix for upside-down correction (rotation of 180 degrees around the X-axis)
    R_flip = np.array([[-1, 0, 0],
                       [0, -1, 0],
                       [0, 0, 1]])

    # Apply the flip to the original rotation matrix
    R_corrected = np.dot(R_flip, R_marker_to_camera)
    tvec_corrected = np.dot(R_flip, tvec)

    # Convert the corrected rotation matrix back to a rotation vector
    rvec_corrected, _ = cv2.Rodrigues(R_corrected)
    return rvec_corrected, tvec_corrected

def draw_pos3d_cv(draw_image, imgpts, filename, connections=None):
    for i, pts in enumerate(imgpts):
        cv2.circle(draw_image, pts, 10, (0, 255, 0), -1)
        cv2.putText(draw_image, f"{i}", pts, cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
        
    for conn in connections_coco3d:
        cv2.line(draw_image, [imgpts[conn[0],0], imgpts[conn[0],1]], 
                [imgpts[conn[1],0], imgpts[conn[1],1]], (255,0,0), 3)
    cv2.imwrite(filename, draw_image)


### Load image and draw the point

In [ ]:
from calibration.utils import draw_axis

In [403]:
img = cv2.imread('./imgs/hs_0001/1727861067542961.png')
img = cv2.flip(img,0)
img = cv2.flip(img,1)
rvec = np.array(pos['rvecs'])
tvec = np.array(pos['tvecs'])
print("Rvec", rvec)
print("Tvec", tvec)
rvec, tvec = vertcal_flip_rvec_tvec(rvec, tvec)
print("Rvec", rvec)
print("Tvec", tvec)
draw_image = draw_axis(img, rvec, tvec, CAMERA_MATRIX, DIST_COEFFS, MARKER_LENGTH)
cv2.imwrite('tmp_hs.png', draw_image)

Rvec [-0.16519393 -2.60398705 -1.45896214]
Tvec [ 658.14184491 -573.62625472 1590.85946295]
Rvec [[ 2.11282809]
 [-0.13403537]
 [ 0.18491364]]
Tvec [-658.14184491  573.62625472 1590.85946295]


True

회전 변환 확인 됨. 

### From Xsense to Camera Image plane

### From Xsense to Camera Image plane

마커:  
+x: 화면 오른쪽  
+y: 화면 안 쪽  
+z: 위쪽   

xsense: 
+x: 화면 바깥쪽 (= -y)  
+y: 화면 오른쪽 (= +x)  
+z: 위쪽 (= z)

In [462]:
def enlarge_pelvis(data):
    # 골반 넓히기
    l_vec = data[:,0] - data[:,15]
    r_vec = data[:,0] - data[:,19]
    data[:,15] = data[:,0] - 1.5*l_vec
    data[:,19] = data[:,0] - 1.5*r_vec
    
    # 엉덩이 내리기
    # h_vec = 0.5*(data[:,1] - data[:,0])
    # data[:,0] -= h_vec
    # data[:,15] -= h_vec
    # data[:,19] -= h_vec
    return data

In [603]:
data = np.load("youngjin.npy")
data = data[3:]
tmp = data[:,:,0].copy()
data[:,:,0] = data[:,:,1]
data[:,:,1] = -tmp

data = enlarge_pelvis(data)

In [604]:
# COCO와 비슷한 모양으로 

coco = np.zeros((len(data), 12, 3))
coco[:,0] = data[:,0]
coco[:,1] = data[:,19] # l hip
coco[:,3] = data[:,15] # r hip
coco[:,2] = data[:,20] # l knee
coco[:,4] = data[:,16] # r knee
coco[:,5] = data[:,6] # head
coco[:,6] = data[:,8] # r shoulder
coco[:,7] = data[:,9] # r elbow
coco[:,8] = data[:,10] # r wrist
coco[:,9] = data[:,12] # l shoulder
coco[:,10] = data[:,13] # l elbow
coco[:,11] = data[:,14] # l wrist

In [627]:
Rot = rotation_matrix(-0, 0, 4)

In [631]:
rvec = np.array(pos['rvecs'])
tvec = np.array(pos['tvecs'])
rvec, tvec = vertcal_flip_rvec_tvec(rvec, tvec)

zero_rotation = np.zeros((3, 1))
zero_translation = np.zeros((3, 1))
R_marker_to_camera, _ = cv2.Rodrigues(rvec)

fixed_3d = np.zeros((len(coco), 12, 3))
fixed_25d = np.zeros((len(coco), 12, 3))

for ind in range(len(coco)):
# for ind in [200, 10000,  20000]:
    P_marker = coco[ind].copy() # np.array([1000,0,0])#data[0][:] # 골반 in marker's coordinate system
    # P_marker *= 1.05
    P_marker[:,0] -= 200
    P_marker[:,1] -= 200
    P_marker[:,2] += 10
    P_marker = np.dot(Rot, P_marker.T).T
    P_camera = np.dot(R_marker_to_camera, P_marker.T).T + tvec.T
    fixed_3d[ind] = P_camera
    
    imgpts, _ = cv2.projectPoints(P_camera, zero_rotation, zero_translation, CAMERA_MATRIX, DIST_COEFFS)
    imgpts = np.int32(imgpts).reshape(-1, 2)
    fixed_25d[ind,:,:2] = imgpts
    fixed_25d[ind,:,2] = P_camera[:,2]

    if ind % 100 == 0:
        #draw_pos3d(P_camera, f"mw_{ind}_3D.png", connections=connections_coco3d)
        img = np.zeros((image_height, image_width, 3), np.uint8)
        draw_pos3d_cv(img, imgpts, f"./cv_yj/cv_yj_{ind}_3D.png", connections=connections_coco3d)
        
    # img = cv2.imread('./imgs/Jemu_0004/1727870427366198.png')
    # img = cv2.flip(img,0)
    # img = cv2.flip(img,1)
    # draw_pos3d_cv(img, imgpts, f"test_{ind}_3D.png", connections=connections_coco3d)


In [632]:
np.save("youngjin_3d.npy", fixed_3d)
np.save("youngjin_25d.npy", fixed_25d)

In [269]:
import matplotlib 
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from mpl_toolkits.mplot3d import Axes3D

In [353]:
def draw_pos3d(data, fn, connections=connections):
    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(111, projection='3d')

    x = data[:, 0]
    y = data[:, 1]
    z = data[:, 2]

    for i in range(len(x)):
        ax.scatter(x[i], y[i], z[i], c='b', marker='o')
        ax.text(x[i], y[i], z[i], f'{i}', color='black')

    for conn in connections:
        ax.plot([x[conn[0]], x[conn[1]]], [y[conn[0]], y[conn[1]]], [z[conn[0]], z[conn[1]]], c='b')

    ax.set_xlim([-200, 1000])
    ax.set_ylim([-1000, 1000])
    ax.set_zlim([-50, 2000])

    ax.set_xlabel('X axis')
    ax.set_ylabel('Y axis')
    ax.set_zlabel('Z axis')

    ax.set_aspect("equal")

    #ax.set_title(f"Frame {frame_idx}")
    #plt.show()
    plt.savefig(fn)
    plt.close()


3차원 좌표에서 골반 너비 조정. 

다른 키포인트도 COCO랑 동일한지 확인 필요

확인용 그림 그려놓고, 바로바로 수정할 수 있도록

